In [1]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

In [2]:
batch_size = 32
epochs = 50
target_size = (256, 256)
folder_name = "testcovid"
directory = f"../Datasets/{folder_name}/Splitted_Dataset"

train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=50,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(rescale=1/255.)
# validation_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
    directory=f"{directory}/train",
    batch_size=batch_size,
    class_mode='categorical',
    target_size=target_size,
    subset="training",
)

test_generator = test_datagen.flow_from_directory(
    directory=f"{directory}/test",
    batch_size=batch_size,
    class_mode="categorical",
    target_size=target_size,
)

validation_generator = train_datagen.flow_from_directory(
    # directory=f"{directory}/val",
    directory=f"{directory}/train",
    batch_size=batch_size,
    class_mode="categorical",
    target_size=target_size,
    subset="validation",
)

Found 544 images belonging to 2 classes.
Found 172 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


# DenseNet Model

In [ ]:
DenseNetBase = DenseNet121(weights='imagenet', include_top=False, input_shape=(256,256,3), classifier_activation='sigmoid')

layer = DenseNetBase.output
layer = GlobalAveragePooling2D()(layer)

layer = Dense(1024, activation='relu', kernel_initializer='he_normal')(layer)

DenseNetPreds = Dense(2, activation='sigmoid')(layer)

DenseNetModel = Model(inputs=DenseNetBase.input, outputs=DenseNetPreds)

print(f"Toplam DenseNet katman sayısı: {len(DenseNetModel.layers)}")

In [ ]:
for layer in DenseNetBase.layers:
    layer.trainable = False

for layer in DenseNetBase.layers[-100:]:
    layer.trainable = True

DenseNetModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
DenseNetModel.summary()

In [ ]:
DenseNetHistory = DenseNetModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=epochs
)

# from keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(
#    monitor="val_accuracy",
#    min_delta=0,
#    patience=5,
#    verbose=1,
#    mode="max",
#    baseline=None,
#    restore_best_weights=True,
#    start_from_epoch=0,
# )
# DenseNetHistory = DenseNetModel.fit(
#     x=train_generator, validation_data=validation_generator, epochs=epochs, callbacks=[early_stopping]
# )

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="max",
    baseline=None,
    restore_best_weights=True,
)
DenseNetHistory2 = DenseNetModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=10, callbacks=[early_stopping]
)

In [ ]:
DenseNetModel.evaluate(test_generator)

# ResNet

In [3]:
ResNetBase = ResNet50(weights='imagenet', include_top=False, input_shape=(256,256,3), classifier_activation='sigmoid')

layer2 = ResNetBase.output

layer2 = GlobalAveragePooling2D()(layer2)

layer2 = Dense(1024, activation='relu')(layer2)

ResNetPreds = Dense(2, activation='sigmoid')(layer2)

ResNetModel = Model(inputs=ResNetBase.input, outputs=ResNetPreds)

print(f"Toplam ResNet katman sayısı: {len(ResNetModel.layers)}")

Toplam ResNet katman sayısı: 178


In [5]:
for layer in ResNetBase.layers:
    layer.trainable = False

for layer in ResNetBase.layers[-10:]:
    layer.trainable = True

ResNetModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

ResNetModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

 conv2_block1_1_conv (Conv2D)   (None, 64, 64, 64)   4160        ['pool1_pool[0][0]']             
                                                                                                  
 conv2_block1_1_bn (BatchNormal  (None, 64, 64, 64)  256         ['conv2_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block1_1_relu (Activatio  (None, 64, 64, 64)  0           ['conv2_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block1_2_conv (Conv2D)   (None, 64, 64, 64)   36928       ['conv2_block1_1_relu[0][0]']    
                                                                                                  
 conv2_blo

In [6]:
ResNetHistory = ResNetModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=epochs
)

Epoch 1/50
17/17 [==============================] - 21s 1s/step - loss: 0.7276 - accuracy: 0.5000 - val_loss: 0.7141 - val_accuracy: 0.5000
Epoch 2/50
17/17 [==============================] - 18s 1s/step - loss: 0.7038 - accuracy: 0.5000 - val_loss: 0.7013 - val_accuracy: 0.5000
Epoch 3/50
17/17 [==============================] - 18s 1s/step - loss: 0.6964 - accuracy: 0.4945 - val_loss: 0.6968 - val_accuracy: 0.5000
Epoch 4/50
10/17 [================>.............] - ETA: 5s - loss: 0.6935 - accuracy: 0.4750

KeyboardInterrupt: 

In [ ]:
ResNetModel.evaluate(test_generator)

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(10, 10))

# plt.suptitle("DenseNet - 100 Katman, ResNet - 10 Katman, Xception - 35 Katman Açık, AdaDelta Optimizer, Özel Epoch")

axs[0, 0].plot(DenseNetHistory.history['accuracy'], label='DenseNet Training Accuracy')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].legend()
axs[0, 0].plot(DenseNetHistory.history['val_accuracy'], label='DenseNet Validation Accuracy')
axs[0, 0].set_title('Validation Accuracy')
axs[0, 0].legend()

axs[0, 1].plot(DenseNetHistory.history['loss'], label='DenseNet Training Loss')
axs[0, 1].set_title('Training Loss')
axs[0, 1].legend()
axs[0, 1].plot(DenseNetHistory.history['val_loss'], label='DenseNet Validation Loss')
axs[0, 1].set_title('Validation Loss')
axs[0, 1].legend()

"""
axs[1, 0].plot(ResNetHistory.history['accuracy'], label='ResNet Training Accuracy')
axs[1, 0].set_title('Training Accuracy')
axs[1, 0].legend()
axs[1, 0].plot(ResNetHistory.history['val_accuracy'], label='ResNet Validation Accuracy')
axs[1, 0].set_title('Validation Accuracy')
axs[1, 0].legend()

axs[1, 1].plot(ResNetHistory.history['loss'], label='ResNet Training Loss')
axs[1, 1].set_title('Training Loss')
axs[1, 1].legend()
axs[1, 1].plot(ResNetHistory.history['val_loss'], label='ResNet Validation Loss')
axs[1, 1].set_title('Validation Loss')
axs[1, 1].legend()
"""

plt.tight_layout()
plt.show()

In [ ]:
import os
from datetime import datetime
from nbconvert import HTMLExporter

folder_path = f"..\\Results\\{folder_name}"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

current_time = datetime.now().strftime("%H.%M_%d.%m.%Y")

html_filename = f"{current_time}.html"
html_file_path = os.path.join(folder_path, html_filename)

notebook_filename = "main_project.ipynb"
exporter = HTMLExporter()
output, _ = exporter.from_filename(notebook_filename)

with open(html_file_path, 'w', encoding='utf-8') as f:
    f.write(output)

print(f"Notebook saved to {folder_path} as {html_filename}")